In [1]:
from pathlib import Path
import sys

import numpy as np
import pyvista as pv
import torch
import vtk
import warp as wp

if sys.path[0] != "..":
    sys.path.insert(0, "..")

device = torch.device("cuda:0")
torch.cuda.device(device)
wp.init()
wp.set_device(str(device))

from modulus.distributed import DistributedManager

DistributedManager.initialize()

Warp 1.0.2 initialized:
   CUDA Toolkit 11.5, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA TITAN RTX" (24 GiB, sm_75, mempool enabled)
   CUDA peer access:
     Not supported
   Kernel cache:
     /home/du/.cache/warp/1.0.2


/data/src/modulus/src/modulus/modulus/distributed/manager.py:346: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(


In [ ]:
import os

if os.environ.get("SLURM_JOB_NAME", None) is None:
    dataset_orig_path = Path("/data/src/modulus/data/drivaer_aws/")
    dataset_part_path = Path("/data/src/modulus/data/drivaer_aws/partitions")
    output_path = dataset_orig_path / f"inference/"
    model_path = Path("/data/src/modulus/models/fignet/drivaerml/lrsoc/model_00999.pth")
    pc_path = Path("/data/src/modulus/data/drivaer_aws/original_pointclouds")
else:
    dataset_orig_path = Path("/lustre/fsw/portfolios/coreai/projects/coreai_modulus_cae/datasets/drivaer_aws/drivaer_data_full")
    dataset_part_path = Path("/lustre/fsw/portfolios/coreai/projects/coreai_modulus_cae/datasets/drivaer_aws/partitions/100_200_400/")
    output_path = Path("/lustre/fsw/portfolios/coreai/users/akamenev/outputs/fignet/drivaerml/inference/")
    model_path = Path("/lustre/fsw/portfolios/coreai/users/akamenev/outputs/fignet/drivaerml/9/model_00999.pth")
    pc_path = Path("/lustre/fsw/portfolios/coreai/projects/coreai_modulus_cae/datasets/drivaer_aws/aero-benchmarking/original_pointclouds/")

In [3]:
import src.data


num_points = 500_000
datamodule = src.data.DrivAerMLDataModule(
    data_path=dataset_part_path,
    num_points=num_points
)

In [4]:
import src.networks
from modulus.models.figconvnet.geometries import GridFeaturesMemoryFormat


model = src.networks.FIGConvUNetDrivAerML(
  aabb_max=[2.0,  1.8,  2.6],
  aabb_min=[-2.0, -1.8, -1.5],
  hidden_channels=[16, 16, 16],
  in_channels=1,
  kernel_size=5,
  mlp_channels=[512, 512], #[2048, 2048],
  neighbor_search_type="radius",
  num_down_blocks=1,
  num_levels=2,
  out_channels=4,
  pooling_layers=[2],
  pooling_type="max",
  reductions=["mean"],
  resolution_memory_format_pairs=[
    (GridFeaturesMemoryFormat.b_xc_y_z, [  5, 150, 100]),
    (GridFeaturesMemoryFormat.b_yc_x_z, [250,   3, 100]),
    (GridFeaturesMemoryFormat.b_zc_x_y, [250, 150,   2]),
  ],
  use_rel_pos_encode=True,
)
# Load checkpoint.
chk = torch.load(model_path)
model.load_state_dict(chk["model"])
model = model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3559.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


FIGConvUNetDrivAerML(
  (point_feature_to_grids): ModuleList(
    (0): Sequential(
      (0): PointFeatureToGrid(
        (conv): PointFeatureConv(in_channels=16 out_channels=16 search_type=radius reductions=['mean'] rel_pos_encode=True)
      )
      (1): GridFeatureMemoryFormatConverter(memory_format=GridFeaturesMemoryFormat.b_xc_y_z)
    )
    (1): Sequential(
      (0): PointFeatureToGrid(
        (conv): PointFeatureConv(in_channels=16 out_channels=16 search_type=radius reductions=['mean'] rel_pos_encode=True)
      )
      (1): GridFeatureMemoryFormatConverter(memory_format=GridFeaturesMemoryFormat.b_yc_x_z)
    )
    (2): Sequential(
      (0): PointFeatureToGrid(
        (conv): PointFeatureConv(in_channels=16 out_channels=16 search_type=radius reductions=['mean'] rel_pos_encode=True)
      )
      (1): GridFeatureMemoryFormatConverter(memory_format=GridFeaturesMemoryFormat.b_zc_x_y)
    )
  )
  (down_blocks): ModuleList(
    (0-1): 2 x Sequential(
      (0): GridFeatureConv2DB

In [5]:
from sklearn.neighbors import NearestNeighbors

from modulus.datapipes.cae.readers import read_vtp


def convert_to_triangular_mesh(
    polydata, write=False, output_filename="surface_mesh_triangular.vtu"
):
    """Converts a vtkPolyData object to a triangular mesh."""
    tet_filter = vtk.vtkDataSetTriangleFilter()
    tet_filter.SetInputData(polydata)
    tet_filter.Update()

    tet_mesh = pv.wrap(tet_filter.GetOutput())

    if write:
        tet_mesh.save(output_filename)

    return tet_mesh


def fetch_mesh_vertices(mesh):
    """Fetches the vertices of a mesh."""
    points = mesh.GetPoints()
    num_points = points.GetNumberOfPoints()
    vertices = [points.GetPoint(i) for i in range(num_points)]
    return vertices

In [6]:
from src.utils.eval_funcs import rrmse

torch.set_grad_enabled(False)

torch.cuda.empty_cache()

@torch.no_grad
def run_inference(is_pointcloud: bool):
    for sample in datamodule.test_dataloader():
        vertices_denorm = model.data_dict_to_input(sample)
        vertices = datamodule.encode(vertices_denorm, "coordinates")
        normalized_pred, _ = model(vertices)
        normalized_p_pred = normalized_pred[..., :1]
        denorm_p_pred = datamodule.decode(normalized_p_pred, "pressure")
        normalized_wss_pred = normalized_pred[..., 1:]
        denorm_wss_pred = datamodule.decode(normalized_wss_pred, "shear_stress")

        # Read the original surface mesh.
        idx = sample["design"][0]
        vtp_file = dataset_orig_path / f"run_{idx}/boundary_{idx}.vtp"
        print(f"Reading {vtp_file}")
        mesh = pv.read(vtp_file)
        mesh = mesh.cell_data_to_point_data()

        # Interpolate predictions on GT mesh.
        k = 4
        nbrs_surface = NearestNeighbors(
            n_neighbors=k, algorithm="ball_tree"
        ).fit(vertices_denorm[0].cpu().numpy())

        distances, indices = nbrs_surface.kneighbors(mesh.points)
        if k == 1:
            indices = indices.flatten()
            pressure_pred_mesh = denorm_p_pred[0][indices]
            shear_stress_pred_mesh = denorm_wss_pred[0][indices]
        else:
            # distances = distances.astype(np.float32)
            # Weighted kNN interpolation
            # Avoid division by zero by adding a small epsilon
            epsilon = 1e-8
            weights = 1 / (distances + epsilon)
            weights_sum = np.sum(weights, axis=1, keepdims=True)
            normalized_weights = weights / weights_sum
            # Fetch the predictions of the k nearest neighbors
            pressure_neighbors = denorm_p_pred[0][indices]  # Shape: (n_samples, k, 1)
            shear_stress_neighbors = denorm_wss_pred[0][indices]  # Shape: (n_samples, k, 3)

            # Compute the weighted average
            pressure_pred_mesh = np.sum(normalized_weights[:, :, np.newaxis] * pressure_neighbors.cpu().numpy(), axis=1)
            shear_stress_pred_mesh = np.sum(normalized_weights[:, :, np.newaxis] * shear_stress_neighbors.cpu().numpy(), axis=1)

            # Convert back to torch tensors
            pressure_pred_mesh = torch.from_numpy(pressure_pred_mesh).to(device)
            shear_stress_pred_mesh = torch.from_numpy(shear_stress_pred_mesh).to(device)

        mesh.point_data["pMeanTrimPred"] = pressure_pred_mesh.cpu().float().numpy()
        mesh.point_data["wallShearStressMeanTrimPred"] = shear_stress_pred_mesh.cpu().float().numpy()
        mesh.save(output_path / f"500K_k4_pc/inference_mesh_{idx}.vtp")
        print("Done.")
        print(
            rrmse(torch.tensor(mesh.point_data["pMeanTrim"]), torch.tensor(mesh.point_data["pMeanTrimPred"])),
            rrmse(torch.tensor(mesh.point_data["wallShearStressMeanTrim"][:, 0]), torch.tensor(mesh.point_data["wallShearStressMeanTrimPred"][:, 0])),
            rrmse(torch.tensor(mesh.point_data["wallShearStressMeanTrim"][:, 1]), torch.tensor(mesh.point_data["wallShearStressMeanTrimPred"][:, 1])),
            rrmse(torch.tensor(mesh.point_data["wallShearStressMeanTrim"][:, 2]), torch.tensor(mesh.point_data["wallShearStressMeanTrimPred"][:, 2])),
        )
        torch.cuda.empty_cache()
        # break

# run_inference()

In [ ]:
@torch.no_grad
def run_inference_on_pc(pc_size: int, k: int = 4):
    for sample in datamodule.test_dataloader():
        vertices_denorm = model.data_dict_to_input(sample)
        vertices = datamodule.encode(vertices_denorm, "coordinates")
        normalized_pred, _ = model(vertices)
        normalized_p_pred = normalized_pred[..., :1]
        denorm_p_pred = datamodule.decode(normalized_p_pred, "pressure")
        normalized_wss_pred = normalized_pred[..., 1:]
        denorm_wss_pred = datamodule.decode(normalized_wss_pred, "shear_stress")

        # Read the original surface mesh.
        idx = sample["design"][0]
        vtp_file = pc_path / f"input_pc_{pc_size}_run_{idx}_final.vtp"
        print(f"Reading {vtp_file}")
        mesh = pv.read(vtp_file)

        # Interpolate predictions on GT mesh.
        nbrs_surface = NearestNeighbors(
            n_neighbors=k, algorithm="ball_tree"
        ).fit(vertices_denorm[0].cpu().numpy())

        distances, indices = nbrs_surface.kneighbors(mesh.points)
        if k == 1:
            indices = indices.flatten()
            pressure_pred_mesh = denorm_p_pred[0][indices]
            shear_stress_pred_mesh = denorm_wss_pred[0][indices]
        else:
            # distances = distances.astype(np.float32)
            # Weighted kNN interpolation
            # Avoid division by zero by adding a small epsilon
            epsilon = 1e-8
            weights = 1 / (distances + epsilon)
            weights_sum = np.sum(weights, axis=1, keepdims=True)
            normalized_weights = weights / weights_sum
            # Fetch the predictions of the k nearest neighbors
            pressure_neighbors = denorm_p_pred[0][indices]  # Shape: (n_samples, k, 1)
            shear_stress_neighbors = denorm_wss_pred[0][indices]  # Shape: (n_samples, k, 3)

            # Compute the weighted average
            pressure_pred_mesh = np.sum(normalized_weights[:, :, np.newaxis] * pressure_neighbors.cpu().numpy(), axis=1)
            shear_stress_pred_mesh = np.sum(normalized_weights[:, :, np.newaxis] * shear_stress_neighbors.cpu().numpy(), axis=1)

            # Convert back to torch tensors
            pressure_pred_mesh = torch.from_numpy(pressure_pred_mesh).to(device)
            shear_stress_pred_mesh = torch.from_numpy(shear_stress_pred_mesh).to(device)

        mesh.point_data["pMeanTrimPred"] = pressure_pred_mesh.cpu().float().numpy()
        mesh.point_data["wallShearStressMeanTrimPred"] = shear_stress_pred_mesh.cpu().float().numpy()
        out_path = output_path / f"pc/500K_k{k}"
        out_path.mkdir(parents=True, exist_ok=True)
        mesh.save(out_path / f"inference_pc_{pc_size}_{idx}.vtp")
        print("Done.")
        torch.cuda.empty_cache()
        # break

run_inference_on_pc(5_000_000)
run_inference_on_pc(10_000_000)
run_inference_on_pc(20_000_000)

Reading /data/src/modulus/data/drivaer_aws/original_pointclouds/input_pc_5000000_run_100_final.vtp


2024-11-20 10:17:40.100 ( 117.435s) [    7F914D57C280]            vtkMath.cxx:778   WARN| vtkMath::Jacobi: Error extracting eigenfunctions
ERROR:root:No data to measure...!
2024-11-20 10:17:40.737 ( 118.071s) [    7F914D57C280]  vtkMassProperties.cxx:60     ERR| vtkMassProperties (0x559237b10530): No data to measure...!


Done.
Reading /data/src/modulus/data/drivaer_aws/original_pointclouds/input_pc_5000000_run_200_final.vtp
Done.
Reading /data/src/modulus/data/drivaer_aws/original_pointclouds/input_pc_5000000_run_300_final.vtp
Done.
Reading /data/src/modulus/data/drivaer_aws/original_pointclouds/input_pc_5000000_run_400_final.vtp
Done.
Reading /data/src/modulus/data/drivaer_aws/original_pointclouds/input_pc_5000000_run_500_final.vtp
Done.


In [ ]:
@torch.no_grad
def inference_on_sim_mesh():
    for idx in [100, 200, 300, 400, 500][:1]:
        mesh_gt = pv.read(dataset_orig_path / f"run_{idx}/boundary_{idx}.vtp")
        mesh_gt = mesh_gt.cell_data_to_point_data()
        step = 500_000 # num_points
        p_chunks = []
        wss_chunks = []
        rng = np.random.default_rng(1)
        indices = rng.permutation(range(mesh_gt.number_of_points))
        for i_start in range(0, mesh_gt.number_of_points, step):
            vertices_denorm = torch.as_tensor(
                mesh_gt.points[indices[i_start : i_start + step]], device=device
            ).unsqueeze(0)
            vertices = datamodule.encode(vertices_denorm, "coordinates")
            normalized_pred, _ = model(vertices)

            normalized_p_pred = normalized_pred[..., :1]
            denorm_p_pred = datamodule.decode(normalized_p_pred, "pressure")
            p_chunks.append(denorm_p_pred.cpu())

            normalized_wss_pred = normalized_pred[..., 1:]
            denorm_wss_pred = datamodule.decode(normalized_wss_pred, "shear_stress")
            wss_chunks.append(denorm_wss_pred.cpu())
            torch.cuda.empty_cache()

        pressure_pred_mesh = torch.cat(p_chunks, dim=1)[0]
        shear_stress_pred_mesh = torch.cat(wss_chunks, dim=1)[0]
        mesh_gt.point_data["pMeanTrimPred"] = pressure_pred_mesh.cpu().float().numpy()
        mesh_gt.point_data["wallShearStressMeanTrimPred"] = shear_stress_pred_mesh.cpu().float().numpy()
        mesh_gt.save(output_path / f"inference_mesh_{idx}.vtp")
        print("Done.")
        torch.cuda.empty_cache()


# inference_on_sim_mesh()

Module modulus.models.figconvnet.warp_neighbor_search load on device 'cuda:0' took 145.78 ms
Done.


In [ ]:
# v = read_vtp(str(dataset_orig_path / f"inference_point_cloud_{idx}.vtp"))
# v = read_vtp(str(dataset_orig_path / f"run_100/boundary_100.vtp"))
# mesh_gt = pv.read(dataset_orig_path / f"run_100/boundary_100.vtp")

# mesh_gt = mesh_gt.cell_data_to_point_data()
# mesh_pred = pv.read(dataset_orig_path / f"inference/inference_mesh_100.vtp")
# rrmse(torch.tensor(mesh_pred.point_data["pMeanTrim"]), torch.tensor(mesh_pred.point_data["pMeanTrimPred"]))

tensor(0.2085)